![alt text](images/uspas.png)

# Concepts of Accelerator Science and Technology 
# Computer Lab: Longitudinal Dynamics
##### Author: N. Neveu, J. Shtalenkova

This session deals with longitudinal dynamics in two sections. The first demonstrates the energy gain or loss of a particle depending on it's arrival time at an RF wave. The second section walks through a particle-in-cell example simulation in Elegant via Sirepo. The Sirepo example is adapted from the [Fundamentals Dispersion lab](https://github.com/uspas/fundamentals-labs/blob/main/Dispersion-Chromaticity.ipynb). 

### Python Notes:

- Highlight (click into) a notebook cell and press Shift+Enter to execute, or use the play button at the top of the window.
- Make sure you execute notebook cells in order. You must re-execute prior cells if you change something needed in a cell above where you are working.
- You can also execute the whole notebook by using 'Run all cells' under the 'Run' tab.
- '2**2' represents two squared, i.e. 2^2
- A colon (:) means all values in that dimension. i.e. array[:,2] = all rows, second column
- You can change the plot limits by adjusting the numbers in 'plt.ylim()'
- Using the '@' symbol does matrix multiplication

----------

In [ ]:
### Run this cell to load needed packages
%matplotlib inline
from ipywidgets import interactive
from matplotlib.widgets import Slider

# Importing plotting library

import matplotlib
from matplotlib import pyplot as plt

#Setting resolution of plot (changes size too)
matplotlib.rcParams['figure.dpi'] = 100

# Importing math library
import numpy as np

# 1. RF Calculations

Energy gained by a particle in an RF cavity can be described by the the $V(t)$ and phase ($\phi$) of the RF waves in the cavity. The frequency ($f$) of the wave is defined by the frequency of the RF cavity. 

$V(t) = V_0\, \text{sin}(2\pi f t) $

$\Delta E = e V(t) = e V_0\, \text{sin} \phi$

Making a lot of assumtions and assuming we are in a circular machine, we can rewrite the energy and phase equations as:

$\Delta E_{n+1} = \Delta E_n + eV (sin \phi_n - sin \phi_s)$

$\phi_{n+1} = \phi_n + \frac{2 \pi h \eta}{E} \Delta E_{n+1}$


Note, the force applied to a particle in an RF cavitiy is described by the Lorentz equation:  
$F = qE + B\times v$

## **Q1.1) Play with the number of steps in the interactive plot. Explain what is happening in the plot.**

In [ ]:
phase = np.linspace(0, np.pi, 1000)

def ideal_energy(phase):
    return -np.cos(phase)

def calc_dE2(dE0, phi0):
    #dE_n+1 = dE_n + eV* ( sin(phi_n) - sin(phi_s) )
    dE  = dE0 + 0.01*(np.sin(phi0) - np.sin(0))
    #phi_n+1 = phi + k * dE_n + eV ( sin(phi_n) - sin(phi_s) )
    phi = phi0 + 0.01*dE 
    return dE, phi

In [ ]:
def dE_plot(steps):
    phi = np.array([np.pi, np.pi, np.pi])/2.0
    delta_energy = np.array([-0.1, 0.0, 0.1])
    dE = ideal_energy(phi)  + delta_energy

    plt.figure()
    plt.plot(phase, ideal_energy(phase), label='Initial RF wave')
    for i in range(steps):
       dE, phi = calc_dE2(dE, phi)    
    #plt.plot(particle_phase, particle_energy, 'o', label='Particle Energy')

    #dE, phi = calc_dE2(particle_phase, particle_energy)
    plt.plot(phi, dE, 'ro', label='Particle Energy')
    plt.xlabel(r'$\theta$')
    plt.ylabel('Energy [arb. units]')
    plt.legend(loc='upper right')
    plt.show()
    
interactive_plot = interactive(dE_plot, steps=(1,50,1))
output = interactive_plot.children[-1]
output.layout.height = '400px'
interactive_plot

## **Q1.2) The next plot is very typical of stable energy gain in a ring. What needs to change to make this plot unstable?**

In [ ]:
particle_phase = np.array([np.pi, np.pi, np.pi])/2.0
delta_energy = np.array([-0.2, 0.0, 0.2])
particle_energy = ideal_energy(particle_phase)  + delta_energy

plt.figure()
#plt.plot(phase, ideal_energy(phase), label='Ideal Energy')
plt.plot(particle_phase, particle_energy, 'o', label='Particle Energy')

dE = particle_energy
phi = particle_phase
for i in range(900):
    dE, phi = calc_dE2(dE, phi)
    plt.plot(phi, dE, 'ro')#, label='Particle Energy')
plt.xlabel('phi')
plt.ylabel('Energy [arb. units]')
#plt.legend(loc='upper right')
plt.show()

# 2. Particle in Cell (PIC) Simulations

### **Q2.1) Dispersion in FODO lattice**

In the transverse lab, we used a FODO beamline without dipoles. Dipoles create dispersion ($\eta$ or D, depending who you talk to), which describes the evolution of transverse size for particles/beams that are off-momentum. Off-momentum particles experience a different force in the dipole than the design orbit, as they have different magnetic rigidity $B\rho=\frac{p}{q}$. 
The initial conditions for this simulation:

| Species  | Election  |   
|---|---| 
| Energy  | 1 GeV  |   
|  X emittance | $\epsilon_x = 6$ mm-mrad  |  
|  Y emittance |  $\epsilon_y = 6$ mm-mrad |  
|  Quadrupole geometric strength | $K = 0.6$ m$^{-2}$|
|  FODO cell length | $L = 5$ m|

Open simulation “FODObeamline-with-dipole” 
> https://www.sirepo.com/elegant#/source/SMHkFUFD


- Look at all the tabs at the top of the page.
- Go to Visualization tab and run the simulation (make sure beamline "FODOcell" is selected). 
- verify that calculated solution is matched. If not, under Tab: Control -> twiss_output, Change Matched to Yes and re-run simulation. 

### **Q2.2) Change the x and/or y offset in the initial distribution. How does this impact the simulation? Paste screen shots here.**

### **Q2.3) Change the initial emittance. How does this impact the simulation? Paste screen shots here.**

### **Q2.4) Why do we need PIC simulations (vs. matrix calculation or single particle dynamics)?**

#### A comment on dipole edge focusing:
First, scroll to the sigma_output plot to observe the effect of the dipole on the beam moments.

Note that the sigma_output plot is generated by tracking a particle distribution through the cell. This distribution does not include a momentum spread, so dispersion should not affect the results of this plot. 

However, you will observe that Sx looks slightly different from the case without a dipole. There is a small de-focusing kick at the entrance of the dipole, and a small focusing kick at the exit of the dipole. These two kicks cancel, and the periodic solution in the horizontal plane is almost identical to the FODO cell without a dipole. 

In contrast, the average value of Sy has decreased significantly, and there is a focusing kick at the entrance and exit. The dipole has a net focusing effect in the vertical plane. 

The reason for these differences is an effect called *edge focusing*, which is caused by fringe fields that extend past the edge of the dipole. An exploration of edge focusing is beyond the scope of this lab, but it is good to be aware that this is another way in which dipoles affect the lattice and beam parameters. Edge focusing is an effect which does not depend on beam energy spread. In contrast, the strength of the effects caused by dispersion and chromaticity do depend on energy spread in the particle distribution - which is mainly determined by the RF system used for acceleration.

#### Effect of dispersion on beam size:

Scroll down and to the twiss_output plot.
Observe dispersion $\eta_x$ (etax). 

hint: to see dispersion function at an appropriate scale, turn off $\beta_x$ and $\beta_y$ curves by un-checking the boxes.